In [1]:
HTML(read(open("style.html"), String))

HTML{String}("<link href='https://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('https://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\", sans-serif;\r\n    font-size: 100%;\r\n}\r\n\r\n</style>")

In [1]:
include("silent.jl")

@silent (macro with 1 method)

In [2]:
using Pkg
import Dates
@silent Pkg.add("NBInclude")
using NBInclude

In [3]:
@nbinclude "2.1 - Board.ipynb"

undomove! (generic function with 1 method)

***

# Gameplay

Während die `Chess.jl` Bibliothek eine grundlegende Implementation des Schachspiels bereitstellt, 

Zu Beginn werden mögliche `GameAction`s und `GameResult`s definiert. Es wurde ein `enum` gewählt, da die Verwendung Fehler verhindert und zukünftige Änderungen erleichtert.

In [4]:
@enum GameAction begin
    Resign
    Undo
end

In [5]:
@enum GameResult begin
    WhiteWin
    BlackWin
    Draw
end

# Gameplay

Die Funktion `play` initialisiert das Schachspiel. Sie ermöglicht es, die Kontrahenten frei zu definieren. Als **Input** benötigt sie Informationen über die Spieler "Schwarz" und "Weiß", sowie das Schachbrett das verwendet werden soll. Valide Spieler für "Schwarz" und "Weiß" sind sowohl `Player()`, als auch die verschiedenen KIs, bsp. `MemoAI(...)`. Sobald das Spiel initialisiert wurde, werden die normalen Schachregeln befolgt.

Hierbei folgt das Spielen eines Schachspiels stets der folgenden Logik:
```
while !isterminal(game)

    move = nextMove(game, whiteTurn ? whitePlayer : blackPlayer)
    domove!(game, move)
    
end
```

**Input**:
+ white &rarr; der weiße Spieler
+ black &rarr; der schwarze Spieler
+ fen &rarr; das zu verwendende Spielbrett im FEN-String-Format (optional, hier wird die Standardstart-Situation verwendet)
+ log &rarr; die zu verwendende Log-Funktion

**Output**:
+ das Schachbrett auf dem gespielt wird

In [4]:
function play(; white, black, fen::String = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -", 
        log::Union{Nothing, Function} = println)::GameResult
    game = Chess.SimpleGame(Chess.fromfen(fen))
    if log != nothing
        logBoard(Chess.board(game), log)
    end
    extboard = ExtendedBoard(Chess.fromfen(fen))
    extundo = ExtendedUndoInfo[]
    while !Chess.isterminal(game)
        answer = getNextMove(Chess.sidetomove(Chess.board(game)) === Chess.WHITE ? white : black, extboard, 
                    log != nothing ? log : function(any) end)
        if answer == Resign
            break
        end
        if answer == Undo
            if game.ply >= 3
                Chess.back!(game)
                undomove!(extboard, pop!(extundo))
                Chess.back!(game)
                undomove!(extboard, pop!(extundo))
            end
            printBoard(Chess.board(game))
            continue
        end
        move = typeof(answer) == Chess.Move ? answer : answer[1]
        Chess.domove!(game, move)
        push!(extundo, domove!(extboard, move))
        if log != nothing
            logBoard(Chess.board(game), log)
            log("Last Move: $(answer)")
        end
    end
    saveGame(game, log)
    return rateGame(game, extboard.repetitionRuleDraw, log)
end

play (generic function with 1 method)

Die Funktion `getNextMove(entity, board, ...)` bildet auf den nächsten durchzuführenden Zug, bzw. ein `GameAction` ab (ggf. können auch zusätzliche Werte zurückgegeben werden). Hierbei wird die tatsächliche Implementation der Funktion über Multiple-Dispatch mit dem `entity` Attribut bestimmt. Dieses wechselt mit der Partei am Zug.

Diese Herangehensweise ermöglicht es uns, dynamisch mehrere AI Generationen hinzuzufügen, ohne die Hauptfunktion der Spieldurchführung zu ändern. Eine Spielseite (Schwarz oder Weiß) ist somit nicht an eine Spieler-Partei (AI oder "echter" Spieler) gebunden. Falls gewünscht, können zwei AIs ohne jegliche Interaktion gegeneinander getestet werden.

Jede Spieler-Partei definiert folgende Komponenten:

- Ein Struct als eigener Datentyp
- Die Funktion `getNextMove` mit der Signatur `getNextMove(<TYPE>, ExtendedBoard, ...) -> (Move ∪ GameAction, ...)`

Damit nicht nur KIs gegeneinander spielen, sondern auch menschliche Spieler ihr Geschick testen können, wird das `struct Player` und die Funktion `getNextMove` definiert. Die Funktion hat folgenden **Input** und **Output**:

**Input**:
+ _::Player &rarr; der Spieler
+ extboard &rarr; das Spielbrett
+ _... &rarr; ein Platzhalter, der es ermöglicht weitere Parameter zu übergeben

**Output**:
+ es wird ein Zug und / oder eine `GameAction` ausgegeben.

Die Züge selbst werden als `String` eingegeben. Dabei ist zu beachten, dass sowohl die Start- als auch die Endposition der bewegten Figur angegeben werden muss. Der Zug eines Bauern von `e2` nach `e4` wird mit `e2e4` beschrieben.

In [7]:
struct Player end

function getNextMove(_::Player, extboard::ExtendedBoard, _...)::Union{Chess.Move, GameAction}
    legalMoves::Chess.MoveList = Chess.moves(extboard.board)
    @assert length(legalMoves) > 0
    while true
        sleep(0.5)
        enteredString = readline()
        if enteredString == "resign" || enteredString == "exit"
            return Resign
        end
        if enteredString == "undo"
            return Undo
        end
        enteredMove = Chess.movefromstring(enteredString)
        if enteredMove ∈ legalMoves
            return enteredMove
        end
        println("Illegal input '$(enteredString)'")
        println("Available actions: resign | exit, undo")
        println("Available moves: $(map(move -> Chess.tostring(move), legalMoves))")
    end
end

getNextMove (generic function with 1 method)

## Weitere Funktionen

Dieser Abschnitt beschreibt Helfer-Funktionen, welche die Hauptfunktion in ihrer Arbeit unterstützen.

Die `logBoard` Funktion wird verwendet, um das Spielbrett visuell im HTML-Format darzustellen.

**Input**:
+ board &rarr; das darzustellende Spielbrett
+ log &rarr; die zu verwendende Log-Funktion

**Output**:
+ das Spielbrett im HTML-Format

In [8]:
function logBoard(board::Chess.Board, log::Function)
    IJulia.clear_output()
    log(Chess.fen(board))
    html = HTML(IJulia.html(board))
    IJulia.display(html)
end

printBoard (generic function with 1 method)

Um das Schachspiel in der Gesamtheit zu bewerten, nutzen wir die Funktion `rateGame`. Sie bekommt als **Input** das komplette Spiel, sowie Informationen über ein Unentschieden nach Wiederholungen, übergeben und gibt als **Output** eine Bewertung aus. Dabei wird zuerst überprüft, ob das Spiel überhaupt beendet ist. Ist das Spiel vorbei, so wird die Endsituation hinsichtlich des Ausgangs des Spiels bewertet. Hier müssen alle möglichen Ausgangsszenarien beachtet werden, d.h. es muss exakt überprüft werden, ob und wenn ja wer gewonnen hat und / oder ob ein Unentschieden vorliegt und wenn ja, welche Regel dies verursacht.

**Input**:
+ game &rarr; das Spiel
+ drawByRepetitionRule &rarr; Kontext der Regel für ein Unentschieden nach Wiederholungen
+ log &rarr; die zu verwendende Log-Funktion

**Output**:
+ ein GameResult, abhängig von der Spiel-Situation

In [3]:
function rateGame(game::Chess.SimpleGame, drawByRepetitionRule::Bool, log::Union{Nothing, Function})::GameResult
    currentBoard = Chess.board(game)
    currentSide = Chess.sidetomove(currentBoard)
    if Chess.isterminal(currentBoard) || drawByRepetitionRule
        if Chess.ischeckmate(currentBoard)
            if log != nothing
                log("Checkmate, $(Chess.coloropp(currentSide)) wins")
            end
            return currentSide == Chess.WHITE ? BlackWin : WhiteWin
        elseif Chess.isstalemate(currentBoard)
            if log != nothing
                log("Draw (Stalemate)")
            end
        elseif Chess.ismaterialdraw(currentBoard)
            if log != nothing
                log("Draw (Material Draw)")
            end
        elseif Chess.isrule50draw(currentBoard)
            if log != nothing
                log("Draw (50 Moves Rule)")
            end
        elseif drawByRepetitionRule
            if log != nothing
                log("Draw (Repetition Rule)")
            end
        else
            if log != nothing
                log("Draw")
            end
        end
    else
        if log != nothing
            log("$(currentSide) resigned, $(Chess.coloropp(currentSide)) wins")
        end
        return currentSide == Chess.WHITE ? BlackWin : WhiteWin
    end
    return Draw
end

LoadError: UndefVarError: Chess not defined

Für Testzwecke und Fehlerfindung, aber auch die spätere Analyse von beendeten Spielen, muss das gesamte Spiel mit allen Zügen gespeichert werden.
Die Funktion `saveGame` wandelt ein Spiel mit sämtlichen Informationen, bsp. den Zügen, in eine PGN-Datei um. `PGN` steht dabei für `Portable Game Notation` und ist Dateiformat für Schachspiele, welches die Partien als lesbaren Text abspeichert.

**Input**:
+ game &rarr; das Spiel
+ log &rarr; die zu verwendende Log-Funktion

**Output**:
+ eine PGN-Datei mit allen Information der Partie.

In [10]:
function saveGame(game::Chess.SimpleGame, log::Union{Nothing, Function})
    try
        mkdir("games")
    catch _ end
    try
        pgnFile = "games/$(replace(string(Dates.now()), ":" => "-")).pgn"
        open(pgnFile, "w") do file
            write(file, Chess.PGN.gametopgn(game))
        end
        if log != nothing       
            log("Saved game to $(pgnFile)")
        end
    catch e
        if log != nothing
            log("Could not save game.")
            log(e)
        end
    end
end

saveGame (generic function with 1 method)

***